In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import ast

from TheMIMIC import *

def save_list_as_pickle(L, given_path, file_name):
    # Ensure the directory exists
    if not os.path.exists(given_path):
        os.makedirs(given_path)
        print(f'\tDirectory created: {given_path}')
    
    # Save the list as a pickle file
    print(f'\tSaving to {given_path}/{file_name}.pkl')
    with open(os.path.join(given_path, f'{file_name}.pkl'), 'wb') as file:
        pickle.dump(L, file)
        
def load_pickle(thePath):
    with open(thePath, 'rb') as f:
        data = pickle.load(f)
    return data

pd.set_option('display.max_colwidth', None)

In [23]:
# Reading part :-(
    
VIY = load_pickle('../../Data/structured/VIY.pkl')
the_visits = [i for i in VIY[:,0]]
VIY.shape

MIMIC = TheMIMIC()
NoteEvents = MIMIC.read_NOTEEVENTS()

NoteEvents['CATEGORY'] = NoteEvents['CATEGORY'].str.strip()

category_distribution = NoteEvents['CATEGORY'].value_counts()
print(category_distribution)

Reading ICUSTAYS.csv file.
|---- Reading complete.
Patients  = 37443
visits    = 46447
ICU stays = 49214
Reading NOTEEVENTS.csv file.


/lustre/home/almusawiaf/PhD_Projects/LoS_ReAdmission/LoS_ReAdmission_Mortality/LoS_ICU/data_generation/unstructured/TheMIMIC.py:24: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_DF = pd.read_csv(f'{self.MIMIC_Path}/{table_name}.csv')


|---- Reading complete.
CATEGORY
Nursing/other        822497
Radiology            522279
Nursing              223556
ECG                  209051
Physician            141624
Discharge summary     59652
Echo                  45794
Respiratory           31739
Nutrition              9418
General                8301
Rehab Services         5431
Social Work            2670
Case Management         967
Pharmacy                103
Consult                  98
Name: count, dtype: int64


In [24]:
NoteEvents = NoteEvents.dropna(subset=['HADM_ID'])
NoteEvents = NoteEvents[NoteEvents["HADM_ID"].isin(the_visits)]

categories_to_keep = ["Nursing/other", "Nursing", "Physician", "Radiology"]
NoteEvents2 = NoteEvents[NoteEvents['CATEGORY'].isin(categories_to_keep)].reset_index(drop=True)

NoteEvents2['CHARTDATE'] = pd.to_datetime(NoteEvents2['CHARTDATE'], format='%Y-%m-%d')
NoteEvents2['CHARTTIME'] = pd.to_datetime(NoteEvents2['CHARTTIME'], format='%Y-%m-%d %H:%M:%S')
NoteEvents2.sort_values(by='CHARTTIME', ascending=True, inplace=True)

new_NoteEvents = NoteEvents2.reset_index(drop=True)

category_distribution = new_NoteEvents['CATEGORY'].value_counts()
print(category_distribution)

CATEGORY
Nursing/other    743959
Radiology        304685
Nursing          183054
Physician        115258
Name: count, dtype: int64


In [27]:
new_NoteEvents.head(1)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,971016,29156,161773.0,2100-06-08,2100-06-08 23:58:00,NaN,Radiology,CHEST (PORTABLE AP),NaN,NaN,"[**2100-6-8**] 11:58\n CHEST (PORTABLE AP) Clip # [**Clip Number (Radiology) 76719**]\n Reason: INTUBATION\n ______________________________________________________________________________\n FINAL REPORT\n STUDY: Portable AP chest x-ray.\n\n INDICATION: 72-year-old female status post intubation.\n\n COMPARISONS: None.\n\n FINDINGS: An endotracheal tube terminates approximately 5 cm superior to the\n carina. A nasogastric tube can be seen with tip beyond the field of view of\n the film. The side port is also beyond the view of the film. The heart is\n normal in size. The mediastinal and hilar contours are unremarkable. The\n aorta is mildly ectatic. There is flattening of the diaphragm consistent with\n mild hyperinflation. Diffuse reticular opacities are present consistent with\n underlying emphysematous change. The soft tissues and osseous structures are\n otherwise unremarkable. No pleural effusion is seen. Biapical disease with\n retraction of the hila and reticular opacities within the lungs. There is\n evidence of prior granulomatous disease.\n\n IMPRESSION: Status post intubation and nasogastric tube placement, both in\n appropriate position. Hyperinflation and reticulonodular opacities consistent\n with underlying emphysema and old granulomatous disease. Clinical correlation\n is recommended. No acute cardiopulmonary disease present.\n\n\n"


In [25]:
needed_cols = ['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'DESCRIPTION', 'TEXT']

NoteEvents_NursingOther = new_NoteEvents[new_NoteEvents['CATEGORY']=="Nursing/other"][needed_cols]
NoteEvents_Nursing      = new_NoteEvents[new_NoteEvents['CATEGORY']=="Nursing"][needed_cols]
NoteEvents_Physician    = new_NoteEvents[new_NoteEvents['CATEGORY']=="Physician"][needed_cols]
NoteEvents_Radiology    = new_NoteEvents[new_NoteEvents['CATEGORY']=="Radiology"][needed_cols]


In [26]:
NoteEvents_Nursing.head(1)

,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,DESCRIPTION,TEXT
110,31585,125380.0,2100-07-03,2100-07-03 03:54:00,Nursing Progress Note,"88M with h/o DM2, HTN, hyperlipidemia, CHF, s/p CVA, fungal bladder\n mass and recurrent UTIs presents from nursing home 3x vomiting this AM,\n now with fever, altered mental status, tremors. Of note was admitted to\n this hospital for PNA 2 weeks ago, discharged with poor swallowing eval\n in nursing home; study on Monday with mild aspiration, has been\n receiving thick liuqids. In ED febrile to 100.9. Received CTX and\n azithromycin. Was noted to have a slight troponin elevation and new TW\n changes in lateral leads, was given ASA. BP initially 80-90s, improved\n with 3L IV NS. Was admitted in [**4-/2100**] for polymicrobial UTI, found to\n have bladder diverticula with fungal mass in bladder, followed by\n urology. Completed tx with abx including short course of diflucan at\n that point. Has chronic indwelling foley at NH, foley replaced in ED\n with some difficulty, + hematuria. UA positive for UTI.\n In ED Foley was changed to cuade and pt put out 250cc of urine.lacte\n down from 3.5 to 1.5\n Neuro status: pt s/p CVA and r side paralysed, has slurred speech and\n hard to understand, but seems oriented in place/in self.\n Pt with poor swallowing eval, gave meds crushed with applesauce\n tolerates good, needs speech and swallow eval on Monday.\n .\n Pneumonia, aspiration\n Assessment:\n CXray shown left side opacity ?pna, no c/o of SOB, no high fever\n Action:\n Blood cx sent, start on Vanco/fluconozol/ceftriaxone\n Response:\n Asymptomatic at this time\n Plan:\n Cont monitoring temp, follow blood cx, please send urine cx\n Urinary tract infection (UTI)\n Assessment:\n Pt passing dark/brown, cloudy urine, u/o dropped to 10cc\n Action:\n Flushed foley\n Response:\n Minimal response after flushed, given fluid bolus. Cont dufflican\n Plan:\n Cont monitoring u/o. possible needs more foley flushed in setting of\n fungal mass in bladder, needs urine cx.\n K 3.4 start repletion with 40meq iv and mag 1.3 repleted with 2gr\n Bladder scan done shown 224 cc, given fluid bolus.\n"


In [ ]:
def keep_first_last(df):
    # Ensure CHARTDATE is in datetime format
    df['CHARTDATE'] = pd.to_datetime(df['CHARTDATE'])

    # Find the earliest CHARTDATE for each HADM_ID
    min_chartdate_per_hadm = df.groupby('HADM_ID')['CHARTDATE'].min().reset_index()

    # Merge with the original DataFrame to keep only rows where CHARTDATE matches the earliest CHARTDATE per HADM_ID
    df_earliest = df.merge(min_chartdate_per_hadm, on=['HADM_ID', 'CHARTDATE'], how='inner')

    # ---------------------------------------------------------------------------------------------------------------
    # Ensure CHARTTIME is in datetime format
    df_earliest['CHARTTIME'] = pd.to_datetime(df_earliest['CHARTTIME'])

    # Sort by HADM_ID and CHARTTIME
    df_earliest = df_earliest.sort_values(by=['HADM_ID', 'CHARTTIME'])

    # Select first and last row per HADM_ID
    df_first = df_earliest.groupby('HADM_ID').head(1)  # First row per HADM_ID
    df_last = df_earliest.groupby('HADM_ID').tail(1)   # Last row per HADM_ID

    # Combine both
    df_final = pd.concat([df_first, df_last]).sort_values(by=['HADM_ID', 'CHARTTIME']).reset_index(drop=True)
    # ---------------------------------------------------------------------------------------------------------------
    # aggregating text into one row.
    df_final = df_final.groupby('HADM_ID', as_index=False).agg({'SUBJECT_ID': 'first', 'TEXT': '\n'.join})
    # ---------------------------------------------------------------------------------------------------------------    
    return df_final

In [ ]:
NE_NursingOther = keep_first_last(NoteEvents_NursingOther)
NE_Nursing      = keep_first_last(NoteEvents_Nursing)
NE_Radiology    = keep_first_last(NoteEvents_Radiology)


In [ ]:
NE_Nursing.head(2)

In [ ]:
# NoteEvents_NursingOther_grouped.to_csv('../../Data/unstructured/text/NursingOther.csv')
# NoteEvents_Nursing_grouped.to_csv('../../Data/unstructured/text/Nursing.csv')
# NoteEvents_Radiology_grouped.to_csv('../../Data/unstructured/text/Radiology.csv')


In [ ]:
NE_NursingOther.head(3)

In [ ]:
# Sample DataFrames
df_nursing       = NE_Nursing
df_nursing_other = NE_NursingOther
df_radiology     = NE_Radiology

# Merge dataframes on HADM_ID with outer join to ensure all records are included
merged_df = df_nursing.merge(df_nursing_other, on='HADM_ID', how='outer', suffixes=('_nursing', '_nursing_other')) \
                      .merge(df_radiology, on='HADM_ID', how='outer')

# Rename columns for clarity
merged_df.rename(columns={'TEXT_nursing': 'Nursing', 'TEXT_nursing_other': 'Nursing/Other', 'TEXT': 'Radiology'}, inplace=True)

# Fill NaN values with empty strings
merged_df.fillna('', inplace=True)

# Combine text fields
merged_df['Combined_Text'] = merged_df.apply(lambda row: 
    f"Nursing:\n{row['Nursing']}.\nNursing/Other:\n{row['Nursing/Other']}.\nRadiology:\n{row['Radiology']}", axis=1)

# Display the final result
final_DF = merged_df[['HADM_ID', 'SUBJECT_ID', 'Combined_Text']]

final_DF = final_DF.rename(columns={'Combined_Text': 'TEXT'})

final_DF.to_csv('../../Data/unstructured/text/ALL_first_last.csv')

In [ ]:
len(final_DF['SUBJECT_ID'].unique())

In [ ]:
len(final_DF['HADM_ID'].unique())

In [ ]:
final_DF.head(3)